In [1]:
%matplotlib inline

from IPython.core.display import display, HTML

from datetime import datetime as dt
from math import log, sqrt

import numpy as np
import pandas as pd

from scipy import linalg
from scipy import stats

from sklearn.utils.extmath import fast_logdet
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error

In [2]:
%run TrainValidTestDataPrep.ipynb
print("Train Set Normality Test")
for c in data_train.columns:
    _stats = stats.jarque_bera(data_train)
    if np.abs(_stats[1] > 0.01): print("%s,%s"%(c, _stats))
print("Valid Set Normality Test")
for c in data_valid.columns:
    _stats = stats.jarque_bera(data_valid[c])
    if np.abs(_stats[1] > 0.01): print("%s,%s"%(c, _stats))
print("Test  Set Normality Test")
for c in data_test.columns:
    _stats = stats.jarque_bera(data_test[c])
    if np.abs(_stats[1] > 0.01): print("%s,%s"%(c, _stats))


Train Set Normality Test
Valid Set Normality Test
0001.HK,(6.064224749785349, 0.048213685104578796)
0857.HK,(8.452645800870712, 0.014605999639945466)
Test  Set Normality Test
0011.HK,(3.2554994473743633, 0.19637096624740646)
0083.HK,(6.814479085838839, 0.03313253519634285)
0386.HK,(1.5497081805806185, 0.4607710070582528)
1928.HK,(4.337978009576364, 0.11429310830561656)
3328.HK,(1.1835132218824163, 0.5533544020994401)
3988.HK,(0.18692085297707353, 0.9107740592407177)
Train Set Normality Test
Valid Set Normality Test
0001.HK,(6.064224749785349, 0.048213685104578796)
0857.HK,(8.452645800870712, 0.014605999639945466)
Test  Set Normality Test
0011.HK,(3.2554994473743633, 0.19637096624740646)
0083.HK,(6.814479085838839, 0.03313253519634285)
0386.HK,(1.5497081805806185, 0.4607710070582528)
1928.HK,(4.337978009576364, 0.11429310830561656)
3328.HK,(1.1835132218824163, 0.5533544020994401)
3988.HK,(0.18692085297707353, 0.9107740592407177)


# Utility functions:

In [3]:
def model_factor_cov_from_transform(model,data_x):
    return np.cov(model.transform(data_x).T)


def model_to_cov_common(model,factor_cov):
    '''pca model -> n x n covariance matrix from common factors '''
    return np.dot(np.dot(model.components_.T,factor_cov),model.components_)


def cov_common_to_corr(cov_common,stdev_marginals):
    corr = cov_common / np.outer(stdev_marginals,stdev_marginals)
    np.fill_diagonal(corr,1.0)
    return corr


def sample_to_stdev_marginals(data_x):
    return np.std(data_x,axis=0)

    
def corr_to_total_cov(corr,stdev_marginals):
    return corr * np.outer(stdev_marginals,stdev_marginals)


def cov_common_to_total_cov(cov_common,data_x):
    '''this is just to check for rounding errors. should be equiv to corr_to_total_cov'''
    cov = cov_common.copy() 
    np.fill_diagonal(cov,np.diagonal(np.cov(data_x.T)))
    return cov
   
   
def performance(model_candidate,data_fit,data_comp):
    model_candidate.fit(data_fit)
    candidate_factor_covar = model_factor_cov_from_transform(model_candidate,data_fit)
    candidate_cov_common = model_to_cov_common(model_candidate,candidate_factor_covar)
    candidate_stdev_marginals = sample_to_stdev_marginals(data_fit)
    candidate_corr = cov_common_to_corr(candidate_cov_common,candidate_stdev_marginals)
    candidate_cov = corr_to_total_cov(candidate_corr,candidate_stdev_marginals)
    candidate_cov_v_sample_cov = np.abs(candidate_cov - np.cov(data_comp.T)).sum()
    return { 'factors':model_candidate.n_components_,
            'cov_tad':candidate_cov_v_sample_cov,
            'cov_tad_pct': candidate_cov_v_sample_cov / np.cov(data_comp.T).sum(),                
             'model': model_log_like_base(candidate_cov,
                                         model_candidate.mean_,
                                         model_candidate.n_features_,
                                         data_comp).mean(),
            'perfect_marginal_foresight': model_log_like_perfect_marginals(candidate_corr,
                                                                           model_candidate.mean_,
                                                                           model_candidate.n_features_,
                                                                           data_comp).mean(),
           'perfect_corr_foresight':model_log_like_perfect_corr(candidate_stdev_marginals,
                                                  model_candidate.mean_,
                                                  model_candidate.n_features_,
                                                  data_comp).mean()}



# Testing/Performance Metrics

The end goal is to be able to forecast the covariance matrix (for optimization or simulation)
I would like to know what you think of the following 'log-liklihood' lifted from the score
function the score function in the scikitlearn pca model.

I think this is starting to move in the right direction as far as test metrics go, but still want to find a way to separate the correl error from the marginal var error.

This is supposed to follow the paper here, which I need to read and come back
http://www.miketipping.com/papers/met-mppca.pdf


In [4]:
def model_log_like_base(model_cov,model_mean,model_n_features,data_x):
    '''this is lifted from PCA.score(X)
    
    this is meant to follow the paper linked here:
     https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/decomposition/pca.py
    
    I copied this out here because I'm having some issue with the PCA classes covar
    and the covar I build myself. Also I want to check variations (what do you think??) 
    '''
    precision = linalg.inv(model_cov)
    Xr = data_x - model_mean
    log_like = -.5 * (Xr * (np.dot(Xr, precision))).sum(axis=1)
    log_like -= .5 * (model_n_features * log(2. * np.pi) - fast_logdet(precision))
    return log_like

#very curious what you think of the below two functions 
#maybe this a good way to separate the effects of correl and marginal var forecast errors?

def model_log_like_perfect_marginals(model_corr,model_mean,model_n_features,data_x):
    # purposely using the sample covar for marginals - hence the name 'perfect marginals'
    stdev_sample_marginals = np.sqrt(np.diagonal(data_x.cov()))
    # reconstruct the cov matrix
    model_cov2 = model_corr * np.outer(stdev_sample_marginals,stdev_sample_marginals)
    return(model_log_like_base(model_cov2,model_mean,model_n_features,data_x))


def model_log_like_perfect_corr(model_stdev_marginals,model_mean,model_n_features,data_x):
    # reconstruct the cov matrix
    sample_corr = data_x.corr()
    model_cov2 =  corr_to_total_cov(sample_corr,model_stdev_marginals)
    return(model_log_like_base(model_cov2,model_mean,model_n_features,data_x))

# Checking SciKitLearn's PCA implementation and testing for stability

Basically exploring the differences in ScikitLearn's PCA package and my hand coded function

I do not really like the PCA class's get_covariance() function. It looks like it returns the common part only
so will use either corr_to_total_cov or cov_common_to_total_cov above.

Note: they are following the paper mentioned above in the log-liklihood function which I need to read.
    In any case it seems I preserve total var better so will stick to my method.
    
Note2: there seems to be some instability here. About 1 in 4 times the assert statement below will fail.

In [5]:
# test the above utils to make sure approximately equal with 
# SciKitLearns internals

#set precision to the 1bps level for var/covar
var_atol = 1e-8
model_to_test = PCA(n_components=5)
model_to_test.fit(data_train)



# from PCA class
pca_class_factor_covar = model_to_test.explained_variance_ * np.eye(model_to_test.n_components_)
pca_class_cov = model_to_test.get_covariance()


# model_factor_covar
factor_covar = model_factor_cov_from_transform(model_to_test,data_train)
# this fails 1 in 4 times or so on my laptop
assert np.isclose(pca_class_factor_covar,factor_covar,atol=var_atol).all()


# model_to_covar_common
cov_common = model_to_cov_common(model_to_test,factor_covar)
mad_cov_common_vs_pca_class_cov = np.abs(cov_common - pca_class_cov).mean()


# cov_common_to_total_cov
stdev_marginals = sample_to_stdev_marginals(data_train)
corr = cov_common_to_corr(cov_common,stdev_marginals)
cov_total1 = corr_to_total_cov(corr,stdev_marginals)
cov_total2 = cov_common_to_total_cov(cov_common,data_train)
sample_cov = np.cov(data_train.T)
mad_pca_class_cov_vs_sample = np.abs(pca_class_cov - sample_cov).mean()
mad_cov_total1_vs_sample_cov = np.abs(cov_total1 - sample_cov).mean()
mad_cov_total2_vs_sample_cov = np.abs(cov_total2 - sample_cov).mean()
mad_cov_total1_vs_cov_total2 = np.abs(cov_total1 - cov_total2).mean()

#


#inspection
{ 'model_exp_var':model_to_test.explained_variance_,
  'model_exp_var minus noise':model_to_test.explained_variance_ - model_to_test.noise_variance_,
  'manual':np.diagonal(factor_covar),
  'mad_cov_common_vs_pca_class_cov':mad_cov_common_vs_pca_class_cov,
 #
  'mad_pca_class_cov_vs_sample':mad_pca_class_cov_vs_sample,
  'mad_cov_total1_vs_sample_cov':mad_cov_total1_vs_sample_cov,
  'mad_cov_total2_vs_sample_cov':mad_cov_total2_vs_sample_cov,
  'mad_cov_total1_vs_cov_total2':mad_cov_total1_vs_cov_total2}

{'model_exp_var': array([4.23872884e-04, 1.05886473e-04, 8.39596749e-05, 6.57204635e-05,
        5.08013443e-05]),
 'model_exp_var minus noise': array([4.10576517e-04, 9.25901060e-05, 7.06633083e-05, 5.24240969e-05,
        3.75049777e-05]),
 'manual': array([4.23872884e-04, 1.05886481e-04, 8.39598271e-05, 6.57214254e-05,
        5.08073277e-05]),
 'mad_cov_common_vs_pca_class_cov': 5.259184742672842e-07,
 'mad_pca_class_cov_vs_sample': 8.452016352472613e-07,
 'mad_cov_total1_vs_sample_cov': 7.896487985615336e-07,
 'mad_cov_total2_vs_sample_cov': 7.894159190984045e-07,
 'mad_cov_total1_vs_cov_total2': 2.328794631291389e-10}

# Train/Validate

1st step is determining the number of factors to use with train & valid

todo: come back to this to determine the best way to recreate train,valid, and test

In [6]:
def pca_generator(data_train,data_valid):
    '''model generator. make model on train with different params and validate'''
    max_factors = np.linalg.matrix_rank(data_train)
    for i in range(max_factors):
        n_factors = i + 1
        model_candidate = PCA(n_components=n_factors)
        model_candidate.fit(data_train)
        yield performance(model_candidate,data_train,data_valid)

In [7]:
# this is just a sanity check
# when the train and valid are the same data,
# by design 'model' should be the same as 'perfect_marginal_foresight' (but we have decent rounding errors)
# and 'perfect_corr_foresight' should be a constant = the 50 factor (full rank model)
# the cov_tad should decrease to zero for the 50 factor model, and so shoudl cov_tad_pct
pd.DataFrame(pca_generator(data_train,data_train))        

,cov_tad,cov_tad_pct,factors,model,perfect_corr_foresight,perfect_marginal_foresight
0,3.048629e-03,0.167540,1,208.726272,212.234807,208.726113
1,2.428187e-03,0.133443,2,209.102690,212.234807,209.102867
2,2.238854e-03,0.123038,3,209.165866,212.234807,209.166151
3,2.066990e-03,0.113593,4,209.147574,212.234807,209.148030
4,1.975414e-03,0.108560,5,209.209360,212.234807,209.209993
5,1.911142e-03,0.105028,6,209.296251,212.234807,209.296987
6,1.809784e-03,0.099458,7,209.444892,212.234807,209.445603
7,1.698171e-03,0.093324,8,209.455089,212.234807,209.455970
8,1.627200e-03,0.089424,9,209.469324,212.234807,209.470327
9,1.483461e-03,0.081525,10,209.689041,212.234807,209.690167


In [8]:
# From this (building the model with data_train and using it on data_valid)
# it seems that up to 2 factors makes sense but probably not more
# note that this changes over time and context. one of the tricks using
# pca is how many factors are appropriate to use on a given day
pd.DataFrame(pca_generator(data_train,data_valid))        

,cov_tad,cov_tad_pct,factors,model,perfect_corr_foresight,perfect_marginal_foresight
0,0.018184,0.495022,1,200.415863,203.620773,202.805759
1,0.018107,0.492908,2,200.211997,203.620773,203.043128
2,0.018157,0.494274,3,199.886114,203.620773,202.851501
3,0.018239,0.496504,4,199.784350,203.620773,202.893440
4,0.018293,0.497990,5,199.603053,203.620773,202.877066
5,0.018355,0.499664,6,199.052828,203.620773,202.499796
6,0.018389,0.500586,7,198.884897,203.620773,202.540199
7,0.018431,0.501743,8,198.407114,203.620773,202.431232
8,0.018462,0.502584,9,198.496151,203.620773,202.539061
9,0.018442,0.502030,10,198.115720,203.620773,202.444598


# Change benchmark: encoding accuracy to encoding forecast accuracy

Assume we can use any data up to the test_data cutoff(s). I'm using data_valid just because its the closest to data_test in time. In practice we can use all the data up to test_data

Although I am interested in the sparce encoding of the return space to describe what has happened over the last hour, days, etc (and can use this as input into the forecast), at the end of the day I really care about the future. 

We are assuming in this PCA model case that we can model the whole market as 5 iid gaussian shocks + 50 individual shocks. I want to test that the covar matrix we produce is the best forecast of the forward covar matrix. And since we can split that into two separate forecasts: the correlations and the marginals, I'm really most interested in the accuracy and stability of the correlation matrix. 

In [9]:
N_FACTORS=2 # now from validation step, see above

model0 = PCA(n_components=N_FACTORS) 
model0 = model0.fit(data_valid) # fitting with data_valid just because its close to train
performance(model0,data_valid,data_test)

{'factors': 2,
 'cov_tad': 0.013307688613307854,
 'cov_tad_pct': 0.515954107558356,
 'model': 199.9243424212995,
 'perfect_marginal_foresight': 201.24610738292097,
 'perfect_corr_foresight': 206.3378506563013}

In [10]:
# this should really be a separate model 
# but lets comp the above to a simple sample covar/correl matrx
# also note, this is how most people make their covar matrix - the simple,naiive way!
# this should be the same as a 50 factor pca model as currently designed

baseline_mean = data_valid.mean()
baseline_cov = np.cov(data_valid.T)
baseline_corr = data_valid.corr()
baseline_stdev_marginals = np.sqrt(np.diagonal(baseline_cov)) # in this case same as pca
baseline_features = np.linalg.matrix_rank(data_valid)

cov_tad_baseline = np.abs(baseline_cov - np.cov(data_test.T)).sum()
{ 'factors':np.nan,
  'cov_tad':cov_tad_baseline,
  'cov_tad_pct': cov_tad_baseline / np.cov(data_test.T).sum(),                
   'model': model_log_like_base(baseline_cov,
                                baseline_mean,                                
                                baseline_features,
                                data_test).mean(),
 'perfect_marginal_foresight': model_log_like_perfect_marginals(baseline_corr,
                                                                baseline_mean,
                                                                baseline_features,
                                                                data_test).mean(),
'perfect_corr_foresight':model_log_like_perfect_corr(baseline_stdev_marginals,
                                                     baseline_mean,
                                                     baseline_features,
                                                     data_test).mean()}


{'factors': nan,
 'cov_tad': 0.012989327704128265,
 'cov_tad_pct': 0.5036108957843023,
 'model': 196.94178608325038,
 'perfect_marginal_foresight': 197.0369753735743,
 'perfect_corr_foresight': 206.33726972671386}

In [11]:
# another way to look at log liklihood - over time 
# going back to train/valid this would be a good way to set up windows maybe

baseline_log_like = model_log_like_base(baseline_cov,
                                baseline_mean,                                
                                baseline_features,
                                data_test)

baseline_log_like.groupby(pd.Grouper(freq='B')).mean()


Unnamed: 0
2018-07-24    204.747167
2018-07-25    204.217117
2018-07-26    209.331416
2018-07-27    206.222979
2018-07-30    211.711891
2018-07-31    205.355864
2018-08-01    200.083791
2018-08-02    202.894896
2018-08-03    196.971652
2018-08-06    180.226590
2018-08-07    189.045730
2018-08-08    197.505094
2018-08-09    194.330078
2018-08-10    205.262286
2018-08-13    200.737798
2018-08-14    192.836520
2018-08-15    190.801233
2018-08-16    175.269108
2018-08-17    192.655482
2018-08-20    189.485981
2018-08-21    187.929742
Freq: B, dtype: float64

# Older stuff - keeping around for now


In [ ]:
#PCA factor covariance matrix 
cov_factor0 = data_train_encoded.cov()

# if the models is iid, the off diagnoal should be close to zero
corr_factor0 = data_train_encoded.corr()
corr_factor0

In [ ]:
# note: i did it this way (factor_exp * covar_common * factor_exp) for illustration
factor_exposures0 = model0.components_
cov_from_common_factors0 = pd.DataFrame(np.dot(np.dot(factor_exposures0.T,cov_factor0),factor_exposures0))

# note: can also do like below, as you can see, they are equivalent
np.isclose(pd.DataFrame(model0.inverse_transform(data_train_encoded)).cov(),cov_from_common_factors0).all()

In [ ]:
#because correlation comes from cross sectional effects only we can create like this (one of many ways):
sample_cov_valid = data_valid.cov()
var_marginals0 = np.diagonal(sample_cov_valid)
stdev_marginals0 = np.sqrt(var_marginals0)
corr0 = cov_from_common_factors0 / np.outer(stdev_marginals0,stdev_marginals0)
np.fill_diagonal(corr0.values,1.0)
cov0 = corr0 * np.outer(stdev_marginals0,stdev_marginals0)

#inspect
pd.DataFrame({'model':model0.get_covariance()[:,0],
              'model_common+sample_marginal':cov0.iloc[:,0],
              'data_valid':sample_cov_valid.iloc[:,0].values})

#todo: why isnt the model an model_common + sample_marginal closer?

In [ ]:
#We can also shave off the specific var for use later as well
var_specific0 = np.diagonal(data_valid.cov()) - np.diagonal(cov_from_common_factors0)


In [ ]:
#inspect
var_breakdown = pd.DataFrame({'marginal':var_marginals0,
                              'specific':var_specific0,
                              'pct':var_specific0/var_marginals0}).set_index(data_valid.columns)
 
#this should be empty    
var_breakdown[var_breakdown['pct']< 0.0]

In [ ]:
log_like0 = model_log_like(cov0,model0.mean_,model0.n_features_,data_test)    
log_like0.groupby(pd.Grouper(freq='B')).mean()

In [ ]:
# this shoudl really be a separate model (and I'll make it one when I have a minute)
# but lets comp the above to a simple sample covar/correl matrx
# also note, this is how most people make their covar matrix - the simple,naiive way!

mean_baseline = data_valid.mean()
cov_baseline = data_valid.cov()
corr_baseline = data_valid.corr()
var_marginals_baseline = np.diagonal(cov_baseline) # in this case same as pca

log_like_baseline = model_log_like(cov_baseline,data_valid.mean(),len(data_valid.columns),data_test)    
log_like_baseline.groupby(pd.Grouper(freq='B')).mean()



In [ ]:
#todo: even though these tests don't make sense, leaving this here
# until I figure out the best way to separate and compare across models
# the cross sectional/corr error from marginal var forecast error

covar_test = data_test.cov()
var_marginals_test = np.diagonal(covar_test)
corr_test = data_test.corr()

#this doesnt necessarily make sense, but low on time so slapping these in here
print("model0")
print("mad corr: {}".format(np.abs(corr_test.values - corr0.values).sum()))
print("mad marginals: {}".format(np.abs(var_marginals_test - var_marginals0).sum()))

print("baseline sample covar")
print("mad corr: {}".format(np.abs(corr_test.values - corr_baseline.values).sum()))
print("mad marginals: {}".format(np.abs(var_marginals_test - var_marginals_baseline).sum()))

In [ ]:
# inspection, how are these correlations?
# i find it somewhat curious that test sample corr is much lower across all 0001HK pairs
# in the pca model this woud be indicative of using too many PC's to reconstruct

corr_test
corr_baseline
df_corr0 = pd.DataFrame(corr0).set_index(corr_test.index)
df_corr0.columns = df_corr0.index

pd.DataFrame({'test sample corr':corr_test['0001.HK'],'baseline':corr_baseline['0001.HK'],'pca0':df_corr0['0001.HK']})

# More todo: 

lets come back to the encoding reconstruction test. I need to think about this a little more.

It is certainly useful for comparing the models, but as I am explaining as we go along I'm really most interested in which of these will give us the most useful correlation and var model for the future rather than which better describes the future (after it has happened).


In [ ]:
data_test_reconst = model0.inverse_transform(data_test_encoded)
#data_test_encoded
#error_p0 = mean_squared_error(data_test_reconst, data_test)

In [ ]:
cov0 = pd.DataFrame(data_test_encoded).cov()
corr0= pd.DataFrame(data_test_encoded).corr()
print(cov0)
print(corr0)